# Notions avancées en Python

On aimerait améliorer notre chargement de données pour analyser uniquemement un pays

Pour cela on va voir quelques notions qu'on a pas encore vu :

- compréhension de liste
- gestion des exceptions
- création de class
- construction d'un module

Et si on a le temps les commandes magiques.

Notre point de départ : un script comme celui de la semaine dernière bricolé

Notre point d'arrivé : cacher un peu tout ça

## 0. Chargement des données & recodage

In [12]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import pyshs
import matplotlib.pyplot as plt
import pyreadstat

def question(v,meta):
    return meta.column_names_to_labels[v]

def modalites(v,meta):
    return meta.value_labels[meta.variable_to_label[v]]

df, meta = pyreadstat.read_sav('./wgm-full-wave2-public-file.sav')

# Sélection de la suisse
data = df[df["COUNTRYNEW"]=="Switzerland"]

data["poids"] = data["WGT"]
data["pays"] = data["COUNTRYNEW"]
data["age"] = data["Age"].fillna("NA")
data["age_reco"] = pd.cut(data['Age'],[0,35,45,55,65,100],
                labels = ["1-[15-35[","2-[35-45[","3-[45-55[","4-[55-65[","5-[65-75]"])
data["genre"] = data["Gender"].replace({1.0: '1-Male', 2.0: '2-Female'})
data["education"] = data["Education"].replace(modalites('Education',meta))
data["revenus"] = data["Household_Income"].replace({1.0: 'Q1', 2.0: 'Q2',3:"Q3",4:"Q4",5:"Q5"})


reco = {1:"1-A lot", 2:"2-Some", 3:"3-Not much/at all",4:"3-Not much/at all",99:"4-NA"}
data["connaissance_science"] = data['W1'].replace(reco)
data["comprendre_science"] = data['W2'].replace(reco)
data["education_science"] = data["W3"].replace(modalites('W3',meta))
data["confiance_hopital"] = data["W4"].replace(reco)
data["confiance_science"] = data["W6"].replace(reco)

### Est-ce qu'un pays est dans la liste ?

Notion de list comprehension

In [6]:
pays = df["COUNTRYNEW"].unique()

In [8]:
pays

array(['United States', 'Egypt', 'Morocco', 'Lebanon', 'Saudi Arabia',
       'Jordan', 'Turkey', 'Indonesia', 'Bangladesh', 'United Kingdom',
       'France', 'Germany', 'Netherlands', 'Belgium', 'Spain', 'Italy',
       'Poland', 'Hungary', 'Czech Republic', 'Romania', 'Sweden',
       'Greece', 'Denmark', 'Iran', 'Hong Kong', 'Japan', 'China',
       'India', 'Venezuela', 'Brazil', 'Mexico', 'Nigeria', 'Kenya',
       'Tanzania', 'Israel', 'Ghana', 'Uganda', 'Benin', 'South Africa',
       'Canada', 'Australia', 'Philippines', 'Sri Lanka', 'Vietnam',
       'Thailand', 'Cambodia', 'Laos', 'Myanmar', 'New Zealand',
       'Ethiopia', 'Mali', 'Senegal', 'Zambia', 'South Korea', 'Taiwan',
       'Georgia', 'Kazakhstan', 'Kyrgyzstan', 'Moldova', 'Russia',
       'Ukraine', 'Burkina Faso', 'Cameroon', 'Zimbabwe', 'Costa Rica',
       'Albania', 'Algeria', 'Argentina', 'Austria', 'Bahrain', 'Bolivia',
       'Bosnia Herzegovina', 'Bulgaria', 'Chile', 'Colombia',
       'Congo Brazzaville'

In [9]:
[i for i in pays if "united" in i.lower()]

['United States', 'United Kingdom', 'United Arab Emirates']

### Gestion des erreurs

SI maintenant on voulait faire un petit script qui prend en entrée le nom d'un pays et renvoie la statistique associée

In [22]:
p = input("Nom du pays :")
print(data[data["COUNTRYNEW"]==p]["confiance_science"].value_counts()).loc["1-A lot"]

Nom du pays :France
Series([], Name: confiance_science, dtype: int64)


AttributeError: 'NoneType' object has no attribute 'loc'

Il peut y avoir des erreurs dans un programme, comment les gérer ?

- Les erreurs ont un type
- il est possible de les "attraper"

In [23]:
p = input("Nom du pays :")
try:
    print(data[data["COUNTRYNEW"]==p]["confiance_science"].value_counts()).loc["1-A lot"]
except:
    print("Un souci dans le code")

Nom du pays :qsqsd
Series([], Name: confiance_science, dtype: int64)
Un souci dans le code


### Améliorer notre code : créer nos objets

Dans le cas précédent on charge un fichier SPSS, on manipule les données et les métadonnées, et souvent on ne s'intéresse qu'à un pays. Est-ce qu'on pourrait faire directement un objet qui fait tout ça ?

In [25]:
class stats_pays:
    def __init__(self,name="Switzerland"):
        df, meta = pyreadstat.read_sav('./wgm-full-wave2-public-file.sav')
        self.data = df[df["COUNTRYNEW"]==name]
        self.meta = meta

On peut rajouter des méthodes à notre objet, par ex nos deux fonctions

In [33]:
class stats_pays:
    def __init__(self,name="Switzerland"):
        df, meta = pyreadstat.read_sav('./wgm-full-wave2-public-file.sav')
        self.data = df[df["COUNTRYNEW"]==name]
        self.meta = meta
        
    def question(self, v):
        return self.meta.column_names_to_labels[v]

    def modalites(self, v):
        return self.meta.value_labels[meta.variable_to_label[v]]

In [34]:
suisse = stats_pays()

In [35]:
suisse.question("W2")

'How Much You Understand the Meaning of Science and Scientists'

On peut intégrer tous les recodages dans notre objet

### Création d'un module

tout le code qu'on a là, on le réutilise de fichier en fichier

On peut le mettre dans un fichier .py et le charger, qui rend disponible notre objet

## Les formules magiques

Un mot sur les magics : https://ipython.readthedocs.io/en/stable/interactive/magics.html

Cas de %time et %timeit pour le temps d'exécution